In [28]:
import numpy as np
import pandas as pd
import gensim 
from gensim import models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import defaultdict
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/nicol/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
queries = pd.read_csv("query_df.csv")
all_queries = queries["query"].values
tags = queries["tag"].values
test_corpus = pd.read_csv("test_corpus.csv")["docs"].values

In [30]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

def d2v_classifier(train_docs:np.ndarray, classes:np.ndarray, test_docs:np.ndarray, topn:int = 5, workers=1, vector_size = 4, epochs=20, dm=1):
    """
    
    """
    
    train_data = pd.DataFrame({"docs": train_docs, "tag":classes})
    train_tagged = train_data.apply(
        lambda r: TaggedDocument(words=tokenize_text(r['docs']), tags=[r.tag]), axis=1)
    
    trainsent = train_tagged.values
    
    doc2vec_model = Doc2Vec(trainsent, workers=workers, vector_size=vector_size, epochs=epochs, dm=dm)
    
    train_targets, train_regressors = zip(
        *[(doc.tags[0], doc2vec_model.infer_vector(doc.words)) for doc in trainsent])
    
    inferred_test_vectors = [doc2vec_model.infer_vector(doc.split()) for doc in test_docs]
    
    knn_test_predictions = [
        doc2vec_model.docvecs.most_similar([pred_vec], topn=topn)[0][0]
        for pred_vec in inferred_test_vectors
    ]
    
    predictions = pd.DataFrame({"doc":test_docs, "class": knn_test_predictions})
    
    return predictions

In [31]:
%%time

d2v_classifier(all_queries, tags, test_corpus)

CPU times: user 2.7 s, sys: 10 ms, total: 2.72 s
Wall time: 2.72 s


/var/folders/42/zns6mz5s5hl_ztjh8th2nnqr0000gn/T/ipykernel_53874/2632625881.py:29: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  doc2vec_model.docvecs.most_similar([pred_vec], topn=topn)[0][0]


,doc,class
0,Correct for summer and the beach,Mixed
1,"Fast service, good quality for price",Delivery Service
2,Always fantastic,Mixed
3,All very well,Product Quality
4,All perfect,Mixed
...,...,...
14661,Everything very ok quality and service of 10 I...,Product Quality
14662,Brilliant! I love these glasses. I have ordere...,Product Quality
14663,The price is good but some glasses arrived wit...,Delivery Service
14664,2 time I order and incredible,Customer Service
